In [2]:
import numpy as np
from tensorflow.keras.models import model_from_json
import operator
import cv2
import sys
import os
import pyttsx3, time



# chargement de model
json_file_alphabet = open("model-bw_alphabet.json", "r")
model_json_alphabet = json_file_alphabet.read()
json_file_alphabet.close()
loaded_model = model_from_json(model_json_alphabet)
# charger et mise a jour des poids (weights)dans le nouveau modèle
loaded_model.load_weights("model-bw_alphabet.h5")
print("Loaded model from disk")

cap = cv2.VideoCapture(0)

# chargement base de données a partir d'un dictionnaire

categories = {'0':'0','AMOUR': 'AMOUR', 'BOIRE': 'BOIRE', 'JETAIME': 'JETAIME','JOUER':'JOUER','MERCI':'MERCI'}

while True:
    _, frame = cap.read()
    # Simulating mirror image
    frame = cv2.flip(frame, 1)

    #voire l'explicaion dans 'collect-data'
    x1 = int(0.3 * frame.shape[1])
    y1 = 10
    x2 = frame.shape[1] - 10
    y2 = int(0.7 * frame.shape[1])
    
    cv2.rectangle(frame, (x1 - 1, y1 - 1), (x2 + 1, y2 + 1), (255, 0, 0), 1)
    # Extractiion de fenetre au niveau gris
    roi = frame[y1:y2, x1:x2]

    
    roi = cv2.resize(roi, (96, 96))
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    _, test_image = cv2.threshold(roi, 120, 255, cv2.THRESH_BINARY)
    cv2.imshow("test", test_image)
    #parcourir dossier de notre base de données
    result = loaded_model.predict(test_image.reshape(1, 96, 96, 1))
    prediction = { 'rien_detecter':[0][0],
                  'AMOUR':result[0][1],
                  'BOIRE':result[0][2],
                  'JETAIME':result[0][3],
                  'JOUER':result[0][4],
                  'MERCI':result[0][5],
                   

                 }
    # Tri basé sur la meilleure prédiction
    prediction = sorted(prediction.items(), key=operator.itemgetter(1), reverse=True)

    # afficher la prediction(resultat en temps reel)
    cv2.putText(frame, prediction[0][0], (10, 120), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 255), 1)
    cv2.imshow("Frame", frame)
   # engine = pyttsx3.init()
   # engine.say(prediction[0][0])
   # engine.runAndWait()
    interrupt = cv2.waitKey(10)
    if interrupt & 0xFF == 27:  # esc key
        break

cap.release()
cv2.destroyAllWindows()


Loaded model from disk
1/1 [==============================] - 0s 29ms/step
